In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import random

%matplotlib inline

os.getcwd()
dis = pd.read_csv("/Users/HudsonCavanagh/dsi-projects/projects/2nd/billboard.csv")
bb = pd.DataFrame(dis)
type(bb)



pandas.core.frame.DataFrame

In [ ]:
#THIS CELL IS A REMINDER THAT DATA TYPES MATTER


# bb.iloc[1:,7:83] = bb.iloc[1:,7:83].apply(pd.to_numeric(errors='coerce'))
# bb['x2nd.week']

# def inter(vals):
#         try:
#             if val[0] != 'x':
#                 pd.to_numeric(s)
#             else: 
#                 continue
#         except ValueError:
#             continue
            
# def inter2(vals):
#     for i in vals:
#         try:
#             if i[0]!= 'x':
#                 int(i)
#             else: 
#                 continue
#         except ValueError:
#             continue
            
#inter(bb.iloc[1:,7:83])
#pd.to_numeric(bb['x2nd.week'], errors='coerce')

#bb = bb_str.iloc[1:,7:83].apply(pd.to_numeric(errors='coerce')) --- this says to_numeric requires at least 1 arguement

#have not been successful in converting this data type, many of the attempts above
#bb = pd.to_numeric(bb_str.iloc[1:,7:83], errors='coerce')

#this interator function doesn't work:
#    
            
#this WORKS, but deletes all other columns:
# bb_foo = bb_str.iloc[1:,7:83].convert_objects(convert_numeric=True)
# bb.iloc[1:,7:83] = bb_foo



#THIS DOESNT WORK
#bb['x2nd.week'] = pd.to_numeric(bb['x2nd.week'], errors='ignore')

#THIS WORKS
#bb['x2nd.week'] = bb['x2nd.week'].convert_objects(convert_numeric=True)
#THIS DOESNT SAVE, need to set bb = (except that overwrites everthing)
# pd.to_numeric(bb.iloc[7:83], errors='ignore') 



# def interator():
#     for i, j in enumerate(bb.columns):
#         if str(j)[0] == 'x':
#             bb[i] = bb[i].convert_objects(convert_numeric=True)
#         else:
#             continue
        
            
# interator() 

# bb.head()


# bb = pd.to_numeric(bb_str.iloc[1:,7:83], errors='ignore')
#bb.iloc[7:83] = bb.iloc[7:83].convert_objects(convert_numeric=True)
#bb.head()

#bb[7:83].apply(pd.convert_objects(convert_numeric=True))

# bb.iloc[7:83] = bb.iloc[7:83].convert_objects(convert_numeric=True)

# print(type(bb['x2nd.week'][3]))

In [2]:
def empty_replace(value):
    if value == '*':
        return np.nan
    else:
        return value      
    
def time_clean1(val):
    return val[0:7].replace(',', ':')


# bb_str.iloc[1:,7:83].apply(pd.convert_objects(convert_numeric=True))
# #print(type(bb['x2nd.week'][2]))

bb['time_on_bb100'] = bb.iloc[:,7:83].count(axis=1)

bb.head()
genres = bb["genre"].value_counts() #genres works
bb["peak_date"] = pd.to_datetime(bb["date.peaked"])
bb["entered_date"] = pd.to_datetime(bb["date.entered"])
bb["time"] = (bb["time"]).apply(time_clean1)
bb["track_length"] = pd.to_timedelta(bb["time"])
bb = bb.applymap(empty_replace)
bb.iloc[:,7:83] = bb.iloc[:,7:83].applymap(lambda x: float(x))

#this threw an error if emptys werent replaced, needed to be indexed with .iloc[:,7:83], 
#and use float applied element-wise wit applymap, and finally needed to be set equal to itself to save inplace

# check if the above worked, if float, then yes! -- print(type(bb['x3rd.week'][3]))

# #data is clean!
#bb.head()

In [3]:
bb['genre'].replace("Rock'n'roll", "Rock", inplace=True)  
bb.groupby(['artist.inverted','genre', 'track'])
bb['weeks_on_bb100'] = bb.iloc[:,7:83].count(axis=1)
bb['avg_on_bb100'] = bb.iloc[:,7:83].mean(axis=1)
bb['peak_rank'] = bb.iloc[:,7:83].apply(np.min,axis=1) #had previously used max, this is returning weirdly  idxmax
bb['worst_rank'] = bb.iloc[:,7:83].max(axis=1) #had previously used max
bb['bb_ratio'] = bb['time_on_bb100']/ (bb['avg_on_bb100'])
#create column that combined avg_on_bb100, and time to convert to $
#can use dollars to evaluate track length, or time on x axis(peak date) to see if things are 'getting better'
bb.drop(bb.columns[7:83], axis=1, inplace=True)
bb.drop(bb.columns[5], axis=1, inplace=True)
bb.drop(bb.columns[3], axis=1, inplace=True)
bb.drop(bb.columns[4], axis=1, inplace=True)
bb.rename(columns={'artist.inverted': 'artist'}, inplace=1) 
bb.head()


,year,artist,track,genre,time_on_bb100,peak_date,entered_date,track_length,weeks_on_bb100,avg_on_bb100,peak_rank,worst_rank,bb_ratio
0,2000,Destiny's Child,Independent Women Part I,Rock,76,2000-11-18,2000-09-23,03:38:00,28,14.821429,1,78,5.127711
1,2000,Santana,"Maria, Maria",Rock,75,2000-04-08,2000-02-12,04:18:00,26,10.500000,1,48,7.142857
2,2000,Savage Garden,I Knew I Loved You,Rock,76,2000-01-29,1999-10-23,04:07:00,33,17.363636,1,71,4.376963
3,2000,Madonna,Music,Rock,73,2000-09-16,2000-08-12,03:45:00,24,13.458333,1,44,5.424149
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),Rock,73,2000-10-14,2000-08-05,03:38:00,21,19.952381,1,57,3.658711


In [ ]:
#before i do this, need to convert all this stuff to ints with iteration


#correct peak for Madonna Music should be 1

In [ ]:
bb.head(10)

In [4]:
#pivot_table by artist so we can see their total time on the chart and rank them by that valuye
#run scatter of song length to how long it was 'big'
art_track_view = pd.pivot_table(bb, index=['genre','artist','track'], values=['peak_rank', 'bb_ratio'], aggfunc=(np.mean), margins=1).sort_values(by="bb_ratio", ascending=0)
print(art_track_view)


#pd.pivot_table(df,index=["Manager","Rep"],values=["Price"],aggfunc=[np.mean,len])

                                                                                bb_ratio  \
genre       artist                       track                                             
Rock        Santana                      Maria, Maria                           7.142857   
            Madonna                      Music                                  5.424149   
            Destiny's Child              Independent Women Part I               5.127711   
            N'Sync                       Bye Bye Bye                            5.118902   
Rap         Elliott, Missy "Misdemeanor" Hot Boyz                               5.093023   
Latin       Iglesias, Enrique            Be With You                            4.605678   
Rock        Aaliyah                      Try Again                              4.562852   
            Savage Garden                I Knew I Loved You                     4.376963   
            Houston, Whitney             My Love Is Your Love                   

In [ ]:
track_view = pd.pivot_table(bb, index=['track'], values=['peak_rank', 'bb_ratio'], aggfunc=(np.mean), margins=1).sort_values(by=['peak_rank'], ascending=1)
print(track_view)
#works! at least for bb_score

In [ ]:
artist_view = pd.pivot_table(bb, index=['artist'], values=['peak_rank', 'bb_ratio'], aggfunc=(np.mean), margins=1).sort_values(by=['bb_ratio'], ascending=0)
print(artist_view)
#works! at least for bb_score

In [ ]:
artist_view_peak = pd.pivot_table(bb, index=['artist'], values=['peak_rank'], aggfunc=[max],margins=1).sort_values(by='peak_rank', ascending=0)
print(artist_view_peak)
#this doesn't work yet, getting a key error for peak_rank??


#aggfunc={"Quantity":len,"Price":[np.sum,np.mean]}

In [ ]:
#could also use length of time on bb pivot

In [ ]:
      
artist_view2 = pd.pivot_table(bb, index=['artist'], values=['peak_rank'], aggfunc=(max), margins=1).sort_values(by=['peak_rank'], ascending=1)
print(artist_view2)

In [ ]:
bb.head(100)

In [ ]:

bb.head()

In [ ]:
#yet to do:
#- highest chart position
#time column to song length
#chart of success v. song track title
#calculate the date each song fell off the charts (date entered + weeks on charts)
#pivot of genre & length/ height on charts
#scatter of time on bb v. avg ranking on bb
#avg ranking v. week 1 (or any other week)



More brainstorming ideas:
    - ttest of 
